In [1]:
import pandas as pd                                                             
from src.data.load_dataset.base_dataset_bi_label import get_X_y_bi              
                                                                                
csv = 'src/tests/test_set_240tweets_labeled_0410.csv' 

In [2]:
df = pd.read_csv(csv)                                                           
                                                                                
X, y = get_X_y_bi(df)     

In [3]:
print(X.shape)                                                                  
print(y.shape)  

(135,)
(135, 3)


In [4]:
!pip install -U sentence-transformers                                           
!pip install -U pytorch_lightning                                               
!pip install -U torchvision     

You should consider upgrading via the '/home/gs/.cache/pypoetry/virtualenvs/sentimentalbb-3QOLceC1-py3.8/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/gs/.cache/pypoetry/virtualenvs/sentimentalbb-3QOLceC1-py3.8/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/gs/.cache/pypoetry/virtualenvs/sentimentalbb-3QOLceC1-py3.8/bin/python -m pip install --upgrade pip' command.


In [5]:
from torch import nn                                                            
                                                                                
class TextClassificationModel(nn.Module):                                       
                                                                                
    def __init__(self, vocab_size, embed_dim, num_class):                       
        super(TextClassificationModel, self).__init__()                         
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)       
        self.fc = nn.Linear(embed_dim, num_class)                               
        self.init_weights()                                                     
                                                                                
    def init_weights(self):                                                     
        initrange = 0.5                                                         
        self.embedding.weight.data.uniform_(-initrange, initrange)              
        self.fc.weight.data.uniform_(-initrange, initrange)                     
        self.fc.bias.data.zero_()                                               
                                                                                
    def forward(self, text, offsets):                                           
        embedded = self.embedding(text, offsets)                                
        return self.fc(embedded)      

In [6]:
def train(model, dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


In [7]:
from typing import Optional, Tuple

import torch
from pytorch_lightning import LightningDataModule
from torch.utils.data import ConcatDataset, DataLoader, Dataset, random_split
from torchvision.datasets import MNIST
from torchvision.transforms import transforms

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
from src.data.load_dataset.base_dataset_bi_label import get_X_y_bi

class OurDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
class MNISTDataModule(LightningDataModule):
    """Example of LightningDataModule for MNIST dataset. A DataModule implements 5 key methods:
        - prepare_data (things to do on 1 GPU/TPU, not on every GPU/TPU in distributed mode)
        - setup (things to do on every accelerator in distributed mode)
        - train_dataloader (the training dataloader)
        - val_dataloader (the validation dataloader(s))
        - test_dataloader (the test dataloader(s))
    This allows you to share a full dataset without explaining how to download,
    split, transform and process the data.
    Read the docs:
        https://pytorch-lightning.readthedocs.io/en/latest/extensions/datamodules.html
    """

    def __init__(self, csv_path="src/tests/test_set_240tweets_labeled_0410.csv"):
        super().__init__()
        self.csv_path = csv_path

    def setup(self, stage: Optional[str] = None):
        """Load data.
        Set variables: `self.data_train`, `self.data_val`, `self.data_test`. This method is
        called by lightning when doing `trainer.fit()` and `trainer.test()`, so be careful
        not to execute the random split twice! The `stage` can be used to differentiate
        whether it's called before trainer.fit()` or `trainer.test()`.
        """

        self.df = pd.read_csv(self.csv_path)
        X, y = get_X_y_bi(df)
        # self.dataset = OurDataset(torch.from_numpy(X), torch.from_numpy(y))
        self.dataset = OurDataset(X, y)
        
        # dataloader = DataLoader(od, batch_size=8)
        size_train = int(len(self.dataset) * 0.8)
        size_test = int(len(self.dataset) * 0.1)
        size_val = len(self.dataset) - (size_test + size_train)
        
        self.data_train, self.data_val, self.data_test = random_split(
            dataset=self.dataset,
            lengths=[size_train, size_val, size_test],
            generator=torch.Generator().manual_seed(42),
        )

    def train_dataloader(self):
        return DataLoader(
            dataset=self.data_train,
            batch_size=8,
            num_workers=1,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.data_val,
            batch_size=8,
            num_workers=1,
            shuffle=True,
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=self.data_test,
            batch_size=8,
            num_workers=1,
            shuffle=True,
        )


In [8]:
from typing import Any, List
import torch
from pytorch_lightning import LightningModule
from torchmetrics import MaxMetric
from torchmetrics.classification.accuracy import Accuracy
from sentence_transformers import SentenceTransformer

class MNISTLitModule(LightningModule):
    """Example of LightningModule for MNIST classification.
    A LightningModule organizes your PyTorch code into 5 sections:
        - Computations (init).
        - Train loop (training_step)
        - Validation loop (validation_step)
        - Test loop (test_step)
        - Optimizers (configure_optimizers)
    Read the docs:
        https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html
    """

    def __init__(self):
        super().__init__()

        # loss function
        # self.criterion = torch.nn.CrossEntropyLoss()
        self.criterion = nn.MSELoss()
        
        self.hparams_ = {
            'weight_decay':0.999,
            'lr':1e-2,
            'embed_dim':384,
            'num_label':3,
        }

        # use separate metric instance for train, val and test step
        # to ensure a proper reduction over the epoch
        # self.fn_acc = Accuracy()
        self.fn_acc = lambda x, y: 42
        #Accuracy()
        self._build_model()
        
    def _build_model(self):
        self.sentence_embed = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        # self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(self.hparams_['embed_dim'], self.hparams_['num_label'])
        self.softmax = nn.Softmax(dim=1)
        # self.init_weights()

    # def forward(self, text, offsets):
    def forward(self, text):
        # print(f"{text = }")
        # print(f"{type(text) = }")
        # print(f"{text.shape = }")
        # print(f"{text.dtype = }")
        # embedded = self.embedding(text, offsets)
        embedded = self.sentence_embed.encode(list(text))
        # print(f"{embedded.shape = }")
        t_embedded = torch.from_numpy(embedded)
        out = self.fc(t_embedded)
        # print(f"{out.shape = }")
        return self.softmax(out)
    
    
    def step(self, batch: Any):
        # print(f"{batch = }")
        
        x, y = batch
        logits = self.forward(x)
        
        logits = logits.to(torch.float)
        y = y.to(torch.float)
        # print(f"{logits.shape = }")
        # print(f"{logits.type() = }")
        # print(f"{y.shape = }")
        # print(f"{y.type() = }")
        # y_flat = torch.argmax(y, dim=1)
        # print(f"{y_flat.shape = }")
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        return loss, preds, y

    def training_step(self, batch: Any, batch_idx: int):
        loss, preds, targets = self.step(batch)

        # log train metrics
        acc = self.fn_acc(preds, targets)
        self.log("train/loss", loss, on_step=False, on_epoch=True, prog_bar=False)
        self.log("train/acc", acc, on_step=False, on_epoch=True, prog_bar=True)

        # we can return here dict with any tensors
        # and then read it in some callback or in `training_epoch_end()`` below
        # remember to always return loss from `training_step()` or else backpropagation will fail!
        return {"loss": loss, "preds": preds, "targets": targets}

    def validation_step(self, batch: Any, batch_idx: int):
        loss, preds, targets = self.step(batch)

        # log val metrics
        acc = self.fn_acc(preds, targets)
        self.log("val/loss", loss, on_step=False, on_epoch=True, prog_bar=False)
        self.log("val/acc", acc, on_step=False, on_epoch=True, prog_bar=True)

        return {"loss": loss, "preds": preds, "targets": targets}

    def validation_epoch_end(self, outputs: List[Any]):
        # acc = self.fn_acc.compute()  # get val accuracy from current epoch
        # self.val_acc_best.update(acc)
        # self.log("val/acc_best", self.val_acc_best.compute(), on_epoch=True, prog_bar=True)
        pass

    def test_step(self, batch: Any, batch_idx: int):
        loss, preds, targets = self.step(batch)

        # log test metrics
        acc = self.test_acc(preds, targets)
        self.log("test/loss", loss, on_step=False, on_epoch=True)
        self.log("test/acc", acc, on_step=False, on_epoch=True)

        return {"loss": loss, "preds": preds, "targets": targets}

    def on_epoch_end(self):
        # reset metrics at the end of every epoch
        # self.fn_acc.reset()
        pass

    def configure_optimizers(self):
        """Choose what optimizers and learning-rate schedulers to use in your optimization.
        Normally you'd need one. But in the case of GANs or similar you might have multiple.
        See examples here:
            https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html#configure-optimizers
        """
        return torch.optim.Adam(
            params=self.parameters(), lr=self.hparams_['lr'], weight_decay=self.hparams_['weight_decay']
        )


In [9]:
from pytorch_lightning import Trainer

trainer = Trainer()

model = MNISTLitModule()
datamodule = MNISTDataModule()

trainer.fit(model=model, datamodule=datamodule)

/home/gs/.cache/pypoetry/virtualenvs/sentimentalbb-3QOLceC1-py3.8/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gs/.cache/pypoetry/virtualenvs/sentimentalbb-3QOLceC1-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1812: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Validation: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Validation: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/gs/.cache/pypoetry/virtualenvs/sentimentalbb-3QOLceC1-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
